<a href="https://colab.research.google.com/github/Abhinav4518/Abhinav4518/blob/main/Model_Selection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV

In [2]:
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier

In [3]:
#loading the data
heart_data=pd.read_csv('/content/heart.csv')


In [4]:
heart_data.head()

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
0,52,1,0,125,212,0,1,168,0,1.0,2,2,3,0
1,53,1,0,140,203,1,0,155,1,3.1,0,0,3,0
2,70,1,0,145,174,0,1,125,1,2.6,0,0,3,0
3,61,1,0,148,203,0,1,161,0,0.0,2,1,3,0
4,62,0,0,138,294,1,1,106,0,1.9,1,3,2,0


In [5]:
heart_data['target'].value_counts()

,count
target,
1,526
0,499


In [6]:
#features and target
X=heart_data.drop(columns='target',axis=1)
Y=heart_data['target']

In [7]:
X=np.asarray(X)
Y=np.asarray(Y)

**Model Selection**

In [9]:
#Comparing the models with default hyperparameter using cross validation
models=[LogisticRegression(max_iter=1000),SVC(kernel='linear'),KNeighborsClassifier(),RandomForestClassifier(random_state=0)]

In [10]:
def compare_models_cross_validation():
  for model in models:

    cv_score=cross_val_score(model,X,Y,cv=5)
    mean_accuracy=sum(cv_score)/len(cv_score)
    mean_accuracy=round(mean_accuracy*100,2)
    print('Cross Validation accuracies for the', model , '=', cv_score)
    print('Accuracy score of the ', model, ' = ',mean_accuracy, ' %')

compare_models_cross_validation()

/usr/local/lib/python3.12/dist-packages/sklearn/linear_model/_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Cross Validation accuracies for the LogisticRegression(max_iter=1000) = [0.88292683 0.85853659 0.87804878 0.82439024 0.80487805]
Accuracy score of the  LogisticRegression(max_iter=1000)  =  84.98  %
Cross Validation accuracies for the SVC(kernel='linear') = [0.88292683 0.86829268 0.84390244 0.81463415 0.80487805]
Accuracy score of the  SVC(kernel='linear')  =  84.29  %
Cross Validation accuracies for the KNeighborsClassifier() = [0.76585366 0.74634146 0.76097561 0.71219512 0.75121951]
Accuracy score of the  KNeighborsClassifier()  =  74.73  %
Cross Validation accuracies for the RandomForestClassifier(random_state=0) = [1.         1.         0.98536585 1.         0.98536585]
Accuracy score of the  RandomForestClassifier(random_state=0)  =  99.41  %


In [11]:
#Comparing the models with different Hyperparameter using GridSearch
models_list=[LogisticRegression(max_iter=10000),SVC(),KNeighborsClassifier(),RandomForestClassifier(random_state=0)]

In [12]:
#Hyperparameter dictionary
model_parameters={
    'log_reg_hyperparameters':{
        'C':[1,5,10,20]
    },
    'svc_hyperparameters':{
        'kernel':['linear','poly','rbf','sigmoid'],
        'C':[1,5,10,20]
    },
    'KNN_hyperparameters':{
        'n_neighbors':[3,5,10]
    },
    'random_forest_hyperparameters':{
        'n_estimators':[10,20,50,100]
    }
}

In [13]:
type(model_parameters)

dict

In [14]:
#Converting keys of Dict into List
model_keys=list(model_parameters.keys())
print(model_keys)

['log_reg_hyperparameters', 'svc_hyperparameters', 'KNN_hyperparameters', 'random_forest_hyperparameters']


In [15]:
model_keys[0]

'log_reg_hyperparameters'

In [17]:
model_parameters[model_keys[0]]

{'C': [1, 5, 10, 20]}

In [23]:
#Applying GridSearchCV
def ModelSelection(list_of_models,hyperparameters_dictionary):
  result=[]
  i=0
  for model in list_of_models:
    key=model_keys[i]
    params=hyperparameters_dictionary[key]
    i+=1
    print(model)
    print(params)

    classifier=GridSearchCV(model,params,cv=5)
    #fitting
    classifier.fit(X,Y)
    result.append({
        'model used':model,
        'highest score':classifier.best_score_,
        'best hyperparameters':classifier.best_params_
    })
  result_dataframe=pd.DataFrame(result,columns=['model used','highest score','best hyperparameters'])

  return result_dataframe

In [24]:
ModelSelection(models_list,model_parameters)

LogisticRegression(max_iter=10000)
{'C': [1, 5, 10, 20]}
SVC()
{'kernel': ['linear', 'poly', 'rbf', 'sigmoid'], 'C': [1, 5, 10, 20]}
KNeighborsClassifier()
{'n_neighbors': [3, 5, 10]}
RandomForestClassifier(random_state=0)
{'n_estimators': [10, 20, 50, 100]}


,model used,highest score,best hyperparameters
0,LogisticRegression(max_iter=10000),0.849756,{'C': 1}
1,SVC(),0.844878,"{'C': 5, 'kernel': 'linear'}"
2,KNeighborsClassifier(),0.890732,{'n_neighbors': 3}
3,RandomForestClassifier(random_state=0),0.997073,{'n_estimators': 20}
